In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/trial1/MMA869 Team Project Team Polygon.ipynb
/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import sklearn as sklearn
import seaborn as sns
from sklearn.preprocessing import StandardScaler

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df = pd.DataFrame()
df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv',header='infer')

In [4]:
df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [5]:
df['EJ_A'] = np.where(df['EJ'] == 'A',1,0)
df['EJ_B'] = np.where(df['EJ'] == 'B',1,0)
df = df.drop(['EJ'],axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 59 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      617 non-null    object 
 1   AB      617 non-null    float64
 2   AF      617 non-null    float64
 3   AH      617 non-null    float64
 4   AM      617 non-null    float64
 5   AR      617 non-null    float64
 6   AX      617 non-null    float64
 7   AY      617 non-null    float64
 8   AZ      617 non-null    float64
 9   BC      617 non-null    float64
 10  BD      617 non-null    float64
 11  BN      617 non-null    float64
 12  BP      617 non-null    float64
 13  BQ      557 non-null    float64
 14  BR      617 non-null    float64
 15  BZ      617 non-null    float64
 16  CB      615 non-null    float64
 17  CC      614 non-null    float64
 18  CD      617 non-null    float64
 19  CF      617 non-null    float64
 20  CH      617 non-null    float64
 21  CL      617 non-null    float64
 22  CR

In [6]:
from sklearn.model_selection import train_test_split

X = df.drop(['Class'],axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

type(X_train)
type(y_train)

pandas.core.series.Series

In [7]:
print(X_train.columns)

Index(['Id', 'AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN',
       'BP', 'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS',
       'CU', 'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
       'EB', 'EE', 'EG', 'EH', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
       'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL', 'EJ_A', 'EJ_B'],
      dtype='object')


In [8]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

imputer = IterativeImputer(estimator=LinearRegression())

X_train = X_train.drop(['Id'],axis=1)
X_test = X_test.drop(['Id'],axis=1)

def rounder(x):
    return round(x,0)

for x in X_train.columns:
    if x == 'EJ_A' or x == 'EJ_B':
        imputer = imputer.fit(X_train[[x]])
        X_train[x + "_Imputed"] = imputer.transform(X_train[[x]])
        X_train = X_train.drop([x],axis=1)
        X_train[x + '_Imputed'] = X_train[x + '_Imputed'].map(rounder)
        
        X_test[x + "_Imputed"] = imputer.transform(X_test[[x]])
        X_test = X_test.drop([x],axis=1)
        X_test[x + '_Imputed'] = X_test[x + '_Imputed'].map(rounder)
    else:
        imputer = imputer.fit(X_train[[x]])
        X_train[x + "_Imputed"] = imputer.transform(X_train[[x]])
        X_train = X_train.drop([x],axis=1)

        X_test[x + "_Imputed"] = imputer.transform(X_test[[x]])
        X_test = X_test.drop([x],axis=1)

In [9]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124 entries, 49 to 181
Data columns (total 57 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   AB_Imputed    124 non-null    float64
 1   AF_Imputed    124 non-null    float64
 2   AH_Imputed    124 non-null    float64
 3   AM_Imputed    124 non-null    float64
 4   AR_Imputed    124 non-null    float64
 5   AX_Imputed    124 non-null    float64
 6   AY_Imputed    124 non-null    float64
 7   AZ_Imputed    124 non-null    float64
 8   BC_Imputed    124 non-null    float64
 9   BD _Imputed   124 non-null    float64
 10  BN_Imputed    124 non-null    float64
 11  BP_Imputed    124 non-null    float64
 12  BQ_Imputed    124 non-null    float64
 13  BR_Imputed    124 non-null    float64
 14  BZ_Imputed    124 non-null    float64
 15  CB_Imputed    124 non-null    float64
 16  CC_Imputed    124 non-null    float64
 17  CD _Imputed   124 non-null    float64
 18  CF_Imputed    124 non-null   

In [10]:
# Model 1: Decision Tree

In [11]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=42, criterion="entropy",
                             min_samples_split=10, min_samples_leaf=10, max_depth=3, max_leaf_nodes=5)
clf.fit(X_train, y_train)

y_pred_dt = clf.predict(X_test)

In [12]:
class_names = [str(x) for x in clf.classes_]

In [13]:
print(clf.tree_.node_count)
print(clf.tree_.impurity)
print(clf.tree_.children_left)
print(clf.tree_.threshold)

9
[0.66319684 0.45298117 0.954434   0.14144054 0.8567986  0.79732651
 0.55862937 0.         0.43055187]
[ 1  3 -1  7  5 -1 -1 -1 -1]
[ 2.28980398  0.49780449 -2.          1.47189504 29.32503986 -2.
 -2.         -2.         -2.        ]


In [14]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_dt)

array([[90, 11],
       [ 5, 18]])

In [15]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_dt, target_names=class_names))

              precision    recall  f1-score   support

           0       0.95      0.89      0.92       101
           1       0.62      0.78      0.69        23

    accuracy                           0.87       124
   macro avg       0.78      0.84      0.81       124
weighted avg       0.89      0.87      0.88       124



In [16]:
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, log_loss

print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred_dt)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, y_pred_dt)))
print("F1 Score = {:.2f}".format(f1_score(y_test, y_pred_dt)))
print("Log Loss = {:.2f}".format(log_loss(y_test, y_pred_dt)))

Accuracy = 0.87
Kappa = 0.61
F1 Score = 0.69
Log Loss = 4.65


In [17]:
# Model 2: Random Forest (Bagging)

In [18]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    random_state=0)

clf_rf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [19]:
y_pred_dt = clf_rf.predict(X_test)

In [20]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_dt)

array([[101,   0],
       [  7,  16]])

In [21]:
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, log_loss

print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred_dt)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, y_pred_dt)))
print("F1 Score = {:.2f}".format(f1_score(y_test, y_pred_dt)))
print("Log Loss = {:.2f}".format(log_loss(y_test, y_pred_dt)))

Accuracy = 0.94
Kappa = 0.79
F1 Score = 0.82
Log Loss = 2.03


In [22]:
import time

In [23]:
# Model 3: LightGBM

In [24]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_validate

# Split the dataframe into X (features) and y (target, aka, label).
X = X_train
y = y_train

params = {
      "cat_l2": 10,
      "cat_smooth": 10,
      "colsample_bytree": 0.8,
      "feature_fraction_bynode":  0.8,
      "learning_rate": 0.03,
      "max_depth": 6,
      "min_child_samples": 63,
      "min_data_per_group": 100,
      "n_estimators": 200,
      "num_leaves": 63,
      "path_smooth": 0,
      "reg_alpha":  0.05,
      "reg_lambda": 0.05,
      "subsample_freq": 1,
      "subsample": 0.8,
      "max_bin": 127,
      "extra_trees": False,
      "is_unbalance": True,
      "boosting_type": 'gbdt',
      "n_jobs": 1,
      "verbosity": -1,
      "seed": 77,
}
        
fit_params= {
    'feature_name': "auto",
}

start = time.time()
estimator = LGBMClassifier(**params)

# The more CV, the better our estimate of the score
inner_cv_scores = cross_validate(estimator, X, y, 
                                 fit_params=fit_params, 
                                 cv=15, 
                                 scoring="roc_auc", 
                                 n_jobs=5, 
                                 verbose=0, 
                                 return_train_score=True)
cv_scores=inner_cv_scores['test_score'].tolist()
duration = time.time() - start

print("CV Scores:")
print(["{:0.4f}".format(cv_score) for cv_score in cv_scores])
print("CV Score mean: {:.4f} ".format(np.mean(cv_scores)))
print("CV Score range: {:0.4f} -- {:0.4f}".format(np.mean(cv_scores) - np.std(cv_scores), np.mean(cv_scores) + np.std(cv_scores)))
print("Fit times: {}".format(duration))
print("Total duration: {}".format(duration))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


CV Scores:
['1.0000', '0.9877', '0.9321', '0.9691', '1.0000', '0.9506', '0.9321', '0.9568', '0.8333', '0.9815', '0.9214', '1.0000', '0.9286', '1.0000', '0.8667']
CV Score mean: 0.9507 
CV Score range: 0.9022 -- 0.9992
Fit times: 6.987669467926025
Total duration: 6.987669467926025


In [25]:
# Of course, at this point, once you've found the best parameters etc., you would retrain the model on the full dataset:
estimator = LGBMClassifier(**params)
estimator = estimator.fit(X, y, **fit_params)

In [26]:
y_pred = estimator.predict(X_test)

In [27]:
print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, y_pred)))
print("F1 Score = {:.2f}".format(f1_score(y_test, y_pred)))
print("Log Loss = {:.2f}".format(log_loss(y_test, y_pred)))

Accuracy = 0.94
Kappa = 0.82
F1 Score = 0.86
Log Loss = 2.03


In [28]:
# Rebalancing

In [29]:
print(y_train.value_counts())
print(y_test.value_counts())

0    408
1     85
Name: Class, dtype: int64
0    101
1     23
Name: Class, dtype: int64


In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

# Helper function
def quick_evaluate_with_dt(X_train, X_test, y_train, y_test, name, balance_weights=False):
    
    cw = None
    if balance_weights == True:
        cw = 'balanced'
        
    clf = DecisionTreeClassifier(random_state=0, class_weight=cw)
    clf = RandomForestClassifier(random_state=0, n_estimators=100, class_weight=cw)
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    accuracy       = accuracy_score(y_test, y_pred)
    f1             = f1_score(y_test, y_pred)
    recall         = recall_score(y_test, y_pred)
    precision      = precision_score(y_test, y_pred)
    roc_auc        = roc_auc_score(y_test, y_pred)
    
    df = pd.DataFrame({"Method"    : [name],
                       "Neg"       : [tn + fn],
                       "True Neg"  : [tn],
                       "False Neg" : [fn],
                       "Pos"       : [tp + fp],
                       "TP"        : [tp],
                       "FP"        : [fp],
                       "Accuracy"  : [accuracy],
                       "Recall"    : [recall],
                       "Precision" : [precision],
                       "F1"        : [f1],
                       "AUC"       : [roc_auc],
                      })
    
    print(df)
    return df

In [31]:
evals_d = list()

In [32]:
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=0).fit_resample(X_train, y_train)

X_resampled.shape
y_resampled.shape
np.bincount(y_resampled)

evals_d.append(quick_evaluate_with_dt(X_resampled, X_test, y_resampled, y_test, 'Over SMOTE'))

       Method  Neg  True Neg  False Neg  Pos  TP  FP  Accuracy    Recall  \
0  Over SMOTE  101        98          3   23  20   3  0.951613  0.869565   

   Precision        F1       AUC  
0   0.869565  0.869565  0.919931  


In [33]:
np.bincount(y_resampled)

array([408, 408])

In [34]:
# Try LGBM after oversampling
X = X_resampled
y = y_resampled

In [35]:
X

,AB_Imputed,AF_Imputed,AH_Imputed,AM_Imputed,AR_Imputed,AX_Imputed,AY_Imputed,AZ_Imputed,BC_Imputed,BD _Imputed,...,FR_Imputed,FS_Imputed,GB_Imputed,GE_Imputed,GF_Imputed,GH_Imputed,GI_Imputed,GL_Imputed,EJ_A_Imputed,EJ_B_Imputed
0,2.179230,5618.942340,85.200147,584.688824,50.253252,7.211226,0.025578,11.835156,1.229900,5022.410130,...,0.497060,0.264147,22.008532,72.611063,10738.557100,27.014460,155.876060,0.001414,0.000000,1.000000
1,0.401662,3863.571140,135.546876,9.340552,8.138688,5.377413,0.025578,10.807930,3.745924,3380.936540,...,1.033560,0.392834,19.515596,103.923176,13.038894,31.643384,28.703520,0.083600,0.000000,1.000000
2,0.452938,2379.684010,92.472162,41.992223,8.138688,7.220085,0.025578,6.302000,1.229900,2211.528130,...,0.497060,0.067730,11.599594,72.611063,762.097329,52.670755,10.840980,0.068062,0.000000,1.000000
3,0.192285,2235.620200,85.200147,18.355919,13.592448,3.924537,0.141288,9.717684,1.229900,4659.520310,...,0.497060,0.074503,18.045880,72.611063,5229.733734,39.278876,44.307844,21.978000,1.000000,0.000000
4,0.170920,1072.870700,90.686817,8.430034,8.138688,4.588962,0.025578,8.665250,1.229900,3596.822345,...,1.021960,0.270920,35.170862,72.611063,7451.651376,20.897136,30.879432,0.257400,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,0.231254,1157.305675,85.200147,8.789707,8.449938,4.330795,0.102921,11.364535,6.461570,5452.380940,...,1.349449,0.619843,16.438411,72.611063,7237.954155,35.765221,73.523631,0.090562,0.000000,1.000000
812,2.386295,11204.537169,85.200147,117.535046,8.138688,29.563681,0.032992,18.495157,41.997729,6434.758490,...,15.001704,0.430274,30.293962,224.923217,9718.111881,63.124822,60.975963,0.174366,0.000000,1.000000
813,0.442951,5335.391507,92.348731,16.575590,8.138688,4.445875,0.025578,9.569706,5.180848,5218.250570,...,0.701091,0.154010,24.984331,158.799399,1183.197470,34.885145,26.239331,0.082197,0.000000,1.000000
814,0.162864,1914.825194,85.200147,18.767565,8.138688,4.415828,0.106280,10.056116,8.251466,4433.036184,...,1.984242,0.100805,18.661461,138.847035,11982.648886,28.952921,23.240936,19.686838,0.895372,0.104628


In [36]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_validate

# Split the dataframe into X (features) and y (target, aka, label).

params = {
      "cat_l2": 10,
      "cat_smooth": 10,
      "colsample_bytree": 0.8,
      "feature_fraction_bynode":  0.8,
      "learning_rate": 0.03,
      "max_depth": 6,
      "min_child_samples": 63,
      "min_data_per_group": 100,
      "n_estimators": 1000,
      "num_leaves": 10,
      "path_smooth": 0,
      "reg_alpha":  0.20,
      "reg_lambda": 0.20,
      "subsample_freq": 1,
      "subsample": 0.8,
      "max_bin": 30,
      "extra_trees": False,
      "is_unbalance": False,
      "boosting_type": 'gbdt',
      "n_jobs": 1,
      "verbosity": -1,
      "seed": 77,
}
        
fit_params= {
    'feature_name': "auto",
}

start = time.time()
estimator = LGBMClassifier(**params)

# The more CV, the better our estimate of the score
inner_cv_scores = cross_validate(estimator, X, y, 
                                 fit_params=fit_params, 
                                 cv=20, 
                                 scoring="roc_auc", 
                                 n_jobs=5, 
                                 verbose=0, 
                                 return_train_score=True)
cv_scores=inner_cv_scores['test_score'].tolist()
duration = time.time() - start

print("CV Scores:")
print(["{:0.4f}".format(cv_score) for cv_score in cv_scores])
print("CV Score mean: {:.4f} ".format(np.mean(cv_scores)))
print("CV Score range: {:0.4f} -- {:0.4f}".format(np.mean(cv_scores) - np.std(cv_scores), np.mean(cv_scores) + np.std(cv_scores)))
print("Fit times: {}".format(duration))
print("Total duration: {}".format(duration))

CV Scores:
['1.0000', '0.9786', '1.0000', '0.9167', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '0.9975']
CV Score mean: 0.9946 
CV Score range: 0.9762 -- 1.0131
Fit times: 12.827705144882202
Total duration: 12.827705144882202


In [37]:
estimator = LGBMClassifier(**params)
estimator = estimator.fit(X, y)

In [38]:
y_pred = estimator.predict(X_test)

In [39]:
print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, y_pred)))
print("F1 Score = {:.2f}".format(f1_score(y_test, y_pred)))
print("Log Loss = {:.2f}".format(log_loss(y_test, y_pred)))

Accuracy = 0.98
Kappa = 0.92
F1 Score = 0.93
Log Loss = 0.87


In [40]:
#Model 10: LGBM Tuned by Optuna

In [41]:
#pip install optuna

In [42]:
import optuna

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

def objective_dt(trial, X, y):

  # Now, define all the hyperparams we want to vary, and what values they are allowed
  # to take.
  #
  # Each trial, optuna will automatically choose values for each hyperparam.
  hyper_params = {
    'max_depth': trial.suggest_int('max_depth',4,32,step=4),
    'reg_alpha': trial.suggest_float('reg_alpha',0.05,0.30,step=0.05),
    'reg_lambda': trial.suggest_float('reg_lambda',0.05,0.30,step=0.05),
    'subsample': trial.suggest_float('subsample',0.2,1.0,step=0.2),
    'max_bin': trial.suggest_int('max_bin',10,130,step=30),
    'subsample_freq': 1,
      'n_estimators': trial.suggest_int('n_estimators',100,1000,step=100),
     'is_unbalance': False,
     'n_jobs': 1,
     'verbosity': -1,
     'seed': 77,
  }
    

  # Use the hyperparams that optuna has chosen for this trial to create a DecisionTreeClassifier
  estimator = LGBMClassifier(**hyper_params)

  # Run CV to see how well these hyper_params do
  cv_scores = cross_val_score(estimator, X, y, cv=15, scoring="f1_macro")
  score = np.mean(cv_scores)

  # Whatever we return here tells optuna how well these parameters did
  return score

study = optuna.create_study(direction="maximize")

study.optimize(lambda trial: objective_dt(trial, X, y), n_trials=100,  gc_after_trial=True)

In [43]:
#study.best_params

In [44]:
#study.best_value

In [45]:
#fig = optuna.visualization.plot_optimization_history(study)
#fig.show()

In [46]:
#fig = optuna.visualization.plot_param_importances(study)
#fig.show()

In [47]:
#estimator = LGBMClassifier(**study.best_params)
#estimator.fit(X,y)

In [48]:
#y_pred = estimator.predict(X_test)

In [49]:
#print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred)))
#print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, y_pred)))
#print("F1 Score = {:.2f}".format(f1_score(y_test, y_pred)))
#print("Log Loss = {:.2f}".format(log_loss(y_test, y_pred)))

In [50]:
X

,AB_Imputed,AF_Imputed,AH_Imputed,AM_Imputed,AR_Imputed,AX_Imputed,AY_Imputed,AZ_Imputed,BC_Imputed,BD _Imputed,...,FR_Imputed,FS_Imputed,GB_Imputed,GE_Imputed,GF_Imputed,GH_Imputed,GI_Imputed,GL_Imputed,EJ_A_Imputed,EJ_B_Imputed
0,2.179230,5618.942340,85.200147,584.688824,50.253252,7.211226,0.025578,11.835156,1.229900,5022.410130,...,0.497060,0.264147,22.008532,72.611063,10738.557100,27.014460,155.876060,0.001414,0.000000,1.000000
1,0.401662,3863.571140,135.546876,9.340552,8.138688,5.377413,0.025578,10.807930,3.745924,3380.936540,...,1.033560,0.392834,19.515596,103.923176,13.038894,31.643384,28.703520,0.083600,0.000000,1.000000
2,0.452938,2379.684010,92.472162,41.992223,8.138688,7.220085,0.025578,6.302000,1.229900,2211.528130,...,0.497060,0.067730,11.599594,72.611063,762.097329,52.670755,10.840980,0.068062,0.000000,1.000000
3,0.192285,2235.620200,85.200147,18.355919,13.592448,3.924537,0.141288,9.717684,1.229900,4659.520310,...,0.497060,0.074503,18.045880,72.611063,5229.733734,39.278876,44.307844,21.978000,1.000000,0.000000
4,0.170920,1072.870700,90.686817,8.430034,8.138688,4.588962,0.025578,8.665250,1.229900,3596.822345,...,1.021960,0.270920,35.170862,72.611063,7451.651376,20.897136,30.879432,0.257400,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,0.231254,1157.305675,85.200147,8.789707,8.449938,4.330795,0.102921,11.364535,6.461570,5452.380940,...,1.349449,0.619843,16.438411,72.611063,7237.954155,35.765221,73.523631,0.090562,0.000000,1.000000
812,2.386295,11204.537169,85.200147,117.535046,8.138688,29.563681,0.032992,18.495157,41.997729,6434.758490,...,15.001704,0.430274,30.293962,224.923217,9718.111881,63.124822,60.975963,0.174366,0.000000,1.000000
813,0.442951,5335.391507,92.348731,16.575590,8.138688,4.445875,0.025578,9.569706,5.180848,5218.250570,...,0.701091,0.154010,24.984331,158.799399,1183.197470,34.885145,26.239331,0.082197,0.000000,1.000000
814,0.162864,1914.825194,85.200147,18.767565,8.138688,4.415828,0.106280,10.056116,8.251466,4433.036184,...,1.984242,0.100805,18.661461,138.847035,11982.648886,28.952921,23.240936,19.686838,0.895372,0.104628


#Feature Engineering
plt.figure(figsize = (20,120))
plotnumber = 1
from scipy.stats import norm
mean, std = norm.fit(X)

for column in X:
    if plotnumber <= 100:
        ax = plt.subplot(28, 3, plotnumber)
        ax.set_title(column)
        xmin,xmax = plt.xlim()
        plt.hist(X[column], bins=50, edgecolor='black',density=True)
        x = np.linspace(xmin,xmax,100)
        p = norm.pdf(x,mean,std)
        plt.plot(x,p,color='black')
        title = "Fit results: mu = %.2f, std = %.2f" % (mean,std)
        plt.title(title + " " + column)
        
    plotnumber = plotnumber + 1

plt.tight_layout()

plt.show()

X['GL_Imputed'].hist()

In [51]:
# Engineer some features to match their apparent distribution

import math

def logger(x):
    if x == 0:
        return 0
    else:
        return math.log(x)

def rooter(x):
    return math.sqrt(x)

X['GF_Imputed_logged'] = X['GF_Imputed'].map(logger)
X['GI_Imputed_logged'] = X['GI_Imputed'].map(logger)
X['FL_Imputed_logged'] = X['FL_Imputed'].map(logger)
X['FE_Imputed_logged'] = X['FE_Imputed'].map(logger)
X['EE_Imputed_logged'] = X['EE_Imputed'].map(logger)
X['EB_Imputed_logged'] = X['EB_Imputed'].map(logger)
X['DI_Imputed_logged'] = X['DI_Imputed'].map(logger)
X['DE_Imputed_logged'] = X['DE_Imputed'].map(logger)
X['CB_Imputed_logged'] = X['CB_Imputed'].map(logger)
X['AM_Imputed_logged'] = X['AM_Imputed'].map(logger)
X['AH_Imputed_logged'] = X['AH_Imputed'].map(logger)
X['BP_Imputed_logged'] = X['BP_Imputed'].map(logger)
X['BR_Imputed_logged'] = X['BR_Imputed'].map(logger)
X['BZ_Imputed_logged'] = X['BZ_Imputed'].map(logger)
X['CC_Imputed_logged'] = X['CC_Imputed'].map(logger)
X['CD_Imputed_logged'] = X['CD _Imputed'].map(logger)
X['CF_Imputed_logged'] = X['CF_Imputed'].map(logger)
X['CH_Imputed_logged'] = X['CH_Imputed'].map(logger)
X['CL_Imputed_logged'] = X['CL_Imputed'].map(logger)
X['DL_Imputed_logged'] = X['DL_Imputed'].map(logger)
X['DU_Imputed_logged'] = X['DU_Imputed'].map(logger)
X['DV_Imputed_logged'] = X['DV_Imputed'].map(logger)
X['DY_Imputed_logged'] = X['DY_Imputed'].map(logger)
X['EG_Imputed_logged'] = X['EG_Imputed'].map(logger)
X['EH_Imputed_logged'] = X['EH_Imputed'].map(logger)
X['EP_Imputed_logged'] = X['EP_Imputed'].map(logger)
X['EU_Imputed_logged'] = X['EU_Imputed'].map(logger)
X['GL_Imputed_binary'] = np.where(X['GL_Imputed'] >= X['GL_Imputed'].mean(),1,0)
X = X.drop('GL_Imputed',axis=1)
X['FC_Imputed_logged'] = X['FC_Imputed'].map(logger)
X['FD_Imputed_logged'] = X['FD _Imputed'].map(logger)
X['FE_Imputed_logged'] = X['FE_Imputed'].map(logger)
X['FL_Imputed_logged'] = X['FL_Imputed'].map(logger)
X['FR_Imputed_logged'] = X['FR_Imputed'].map(logger)
X['FS_Imputed_logged'] = X['FS_Imputed'].map(logger)
X['FL_Imputed_logged'] = X['FL_Imputed'].map(logger)
X['GE_Imputed_logged'] = X['GE_Imputed'].map(logger)
X['GF_Imputed_logged'] = X['GF_Imputed'].map(logger)
X['GI_Imputed_logged'] = X['GI_Imputed'].map(logger)
X['GB_Imputed_logged'] = X['GB_Imputed'].map(logger)

X['AX_Imputed_rooted'] = X['AX_Imputed'].map(rooter)
X['AZ_Imputed_rooted'] = X['AZ_Imputed'].map(rooter)
X['BD_Imputed_rooted'] = X['BD _Imputed'].map(rooter)
X['BN_Imputed_rooted'] = X['BN_Imputed'].map(rooter)
X['CR_Imputed_rooted'] = X['CR_Imputed'].map(rooter)
X['CS_Imputed_rooted'] = X['CS_Imputed'].map(rooter)
X['CU_Imputed_rooted'] = X['CU_Imputed'].map(rooter)
X['DA_Imputed_rooted'] = X['DA_Imputed'].map(rooter)
X['DH_Imputed_rooted'] = X['DH_Imputed'].map(rooter)
X['DN_Imputed_rooted'] = X['DN_Imputed'].map(rooter)
X['FI_Imputed_rooted'] = X['FI_Imputed'].map(rooter)
X['GH_Imputed_rooted'] = X['GH_Imputed'].map(rooter)

In [52]:
# May want to try engineering some interaction features with highly correlated features

from sklearn.preprocessing import PowerTransformer
scaler = PowerTransformer(method='box-cox')

def squared(x):
    return x**2 + 1

for column in X:
    y = X[column].map(squared)
    y = y.values.reshape(-1,1)
    scaler.fit(y)
    X[column +'_box_cox'] = scaler.transform(y)

X

In [53]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_validate

# Split the dataframe into X (features) and y (target, aka, label).

params = {
      "cat_l2": 10,
      "cat_smooth": 10,
      "colsample_bytree": 0.8,
      "feature_fraction_bynode":  0.8,
      "learning_rate": 0.01,
      "max_depth": 6,
      "min_child_samples": 63,
      "min_data_per_group": 100,
      "n_estimators": 1000,
      "num_leaves": 10,
      "path_smooth": 0,
      "reg_alpha":  0.30,
      "reg_lambda": 0.30,
      "subsample_freq": 1,
      "subsample": 0.8,
      "max_bin": 30,
      "extra_trees": False,
      "is_unbalance": False,
      "boosting_type": 'gbdt',
      "n_jobs": 1,
      "verbosity": -1,
      "seed": 77,
}
        
fit_params= {
    'feature_name': "auto",
}

start = time.time()
estimator = LGBMClassifier(**params)

# The more CV, the better our estimate of the score
inner_cv_scores = cross_validate(estimator, X, y, 
                                 fit_params=fit_params, 
                                 cv=20, 
                                 scoring="roc_auc", 
                                 n_jobs=5, 
                                 verbose=0, 
                                 return_train_score=True)
cv_scores=inner_cv_scores['test_score'].tolist()
duration = time.time() - start

print("CV Scores:")
print(["{:0.4f}".format(cv_score) for cv_score in cv_scores])
print("CV Score mean: {:.4f} ".format(np.mean(cv_scores)))
print("CV Score range: {:0.4f} -- {:0.4f}".format(np.mean(cv_scores) - np.std(cv_scores), np.mean(cv_scores) + np.std(cv_scores)))
print("Fit times: {}".format(duration))
print("Total duration: {}".format(duration))

CV Scores:
['1.0000', '0.9643', '0.9976', '0.8905', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '1.0000', '0.9900']
CV Score mean: 0.9921 
CV Score range: 0.9675 -- 1.0168
Fit times: 21.389976024627686
Total duration: 21.389976024627686


In [54]:
estimator = LGBMClassifier(**params)
estimator = estimator.fit(X, y)

In [55]:
X_test.columns.unique()

Index(['AB_Imputed', 'AF_Imputed', 'AH_Imputed', 'AM_Imputed', 'AR_Imputed',
       'AX_Imputed', 'AY_Imputed', 'AZ_Imputed', 'BC_Imputed', 'BD _Imputed',
       'BN_Imputed', 'BP_Imputed', 'BQ_Imputed', 'BR_Imputed', 'BZ_Imputed',
       'CB_Imputed', 'CC_Imputed', 'CD _Imputed', 'CF_Imputed', 'CH_Imputed',
       'CL_Imputed', 'CR_Imputed', 'CS_Imputed', 'CU_Imputed', 'CW _Imputed',
       'DA_Imputed', 'DE_Imputed', 'DF_Imputed', 'DH_Imputed', 'DI_Imputed',
       'DL_Imputed', 'DN_Imputed', 'DU_Imputed', 'DV_Imputed', 'DY_Imputed',
       'EB_Imputed', 'EE_Imputed', 'EG_Imputed', 'EH_Imputed', 'EL_Imputed',
       'EP_Imputed', 'EU_Imputed', 'FC_Imputed', 'FD _Imputed', 'FE_Imputed',
       'FI_Imputed', 'FL_Imputed', 'FR_Imputed', 'FS_Imputed', 'GB_Imputed',
       'GE_Imputed', 'GF_Imputed', 'GH_Imputed', 'GI_Imputed', 'GL_Imputed',
       'EJ_A_Imputed', 'EJ_B_Imputed'],
      dtype='object')

In [56]:
X_test['GF_Imputed_logged'] = X_test['GF_Imputed'].map(logger)
X_test['GI_Imputed_logged'] = X_test['GI_Imputed'].map(logger)
X_test['FL_Imputed_logged'] = X_test['FL_Imputed'].map(logger)
X_test['FE_Imputed_logged'] = X_test['FE_Imputed'].map(logger)
X_test['EE_Imputed_logged'] = X_test['EE_Imputed'].map(logger)
X_test['EB_Imputed_logged'] = X_test['EB_Imputed'].map(logger)
X_test['DI_Imputed_logged'] = X_test['DI_Imputed'].map(logger)
X_test['DE_Imputed_logged'] = X_test['DE_Imputed'].map(logger)
X_test['CB_Imputed_logged'] = X_test['CB_Imputed'].map(logger)
X_test['AM_Imputed_logged'] = X_test['AM_Imputed'].map(logger)
X_test['AH_Imputed_logged'] = X_test['AH_Imputed'].map(logger)
X_test['BP_Imputed_logged'] = X_test['BP_Imputed'].map(logger)
X_test['BR_Imputed_logged'] = X_test['BR_Imputed'].map(logger)
X_test['BZ_Imputed_logged'] = X_test['BZ_Imputed'].map(logger)
X_test['CC_Imputed_logged'] = X_test['CC_Imputed'].map(logger)
X_test['CD_Imputed_logged'] = X_test['CD _Imputed'].map(logger)
X_test['CF_Imputed_logged'] = X_test['CF_Imputed'].map(logger)
X_test['CH_Imputed_logged'] = X_test['CH_Imputed'].map(logger)
X_test['CL_Imputed_logged'] = X_test['CL_Imputed'].map(logger)
X_test['DL_Imputed_logged'] = X_test['DL_Imputed'].map(logger)
X_test['DU_Imputed_logged'] = X_test['DU_Imputed'].map(logger)
X_test['DV_Imputed_logged'] = X_test['DV_Imputed'].map(logger)
X_test['DY_Imputed_logged'] = X_test['DY_Imputed'].map(logger)
X_test['EG_Imputed_logged'] = X_test['EG_Imputed'].map(logger)
X_test['EH_Imputed_logged'] = X_test['EH_Imputed'].map(logger)
X_test['EP_Imputed_logged'] = X_test['EP_Imputed'].map(logger)
X_test['EU_Imputed_logged'] = X_test['EU_Imputed'].map(logger)
X_test['GL_Imputed_binary'] = np.where(X_test['GL_Imputed'] >= X_test['GL_Imputed'].mean(),1,0)
X_test = X_test.drop('GL_Imputed',axis=1)
X_test['FC_Imputed_logged'] = X_test['FC_Imputed'].map(logger)
X_test['FD_Imputed_logged'] = X_test['FD _Imputed'].map(logger)
X_test['FE_Imputed_logged'] = X_test['FE_Imputed'].map(logger)
X_test['FL_Imputed_logged'] = X_test['FL_Imputed'].map(logger)
X_test['FR_Imputed_logged'] = X_test['FR_Imputed'].map(logger)
X_test['FS_Imputed_logged'] = X_test['FS_Imputed'].map(logger)
X_test['FL_Imputed_logged'] = X_test['FL_Imputed'].map(logger)
X_test['GE_Imputed_logged'] = X_test['GE_Imputed'].map(logger)
X_test['GF_Imputed_logged'] = X_test['GF_Imputed'].map(logger)
X_test['GI_Imputed_logged'] = X_test['GI_Imputed'].map(logger)
X_test['GB_Imputed_logged'] = X_test['GB_Imputed'].map(logger)

X_test['AX_Imputed_rooted'] = X_test['AX_Imputed'].map(rooter)
X_test['AZ_Imputed_rooted'] = X_test['AZ_Imputed'].map(rooter)
X_test['BD_Imputed_rooted'] = X_test['BD _Imputed'].map(rooter)
X_test['BN_Imputed_rooted'] = X_test['BN_Imputed'].map(rooter)
X_test['CR_Imputed_rooted'] = X_test['CR_Imputed'].map(rooter)
X_test['CS_Imputed_rooted'] = X_test['CS_Imputed'].map(rooter)
X_test['CU_Imputed_rooted'] = X_test['CU_Imputed'].map(rooter)
X_test['DA_Imputed_rooted'] = X_test['DA_Imputed'].map(rooter)
X_test['DH_Imputed_rooted'] = X_test['DH_Imputed'].map(rooter)
X_test['DN_Imputed_rooted'] = X_test['DN_Imputed'].map(rooter)
X_test['FI_Imputed_rooted'] = X_test['FI_Imputed'].map(rooter)
X_test['GH_Imputed_rooted'] = X_test['GH_Imputed'].map(rooter)

In [57]:
y_pred = estimator.predict(X_test)

In [58]:
print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, y_pred)))
print("F1 Score = {:.2f}".format(f1_score(y_test, y_pred)))
print("Log Loss = {:.2f}".format(log_loss(y_test, y_pred)))

Accuracy = 0.99
Kappa = 0.97
F1 Score = 0.98
Log Loss = 0.29


In [59]:
confusion_matrix(y_test, y_pred)

array([[101,   0],
       [  1,  22]])

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_validate

# Split the dataframe into X (features) and y (target, aka, label).

params = {
      "cat_l2": 10,
      "cat_smooth": 10,
      "colsample_bytree": 0.8,
      "feature_fraction_bynode":  0.8,
      "learning_rate": 0.01,
      "max_depth": 6,
      "min_child_samples": 63,
      "min_data_per_group": 100,
      "n_estimators": 1000,
      "num_leaves": 10,
      "path_smooth": 0,
      "reg_alpha":  0.30,
      "reg_lambda": 0.30,
      "subsample_freq": 1,
      "subsample": 0.8,
      "max_bin": 30,
      "extra_trees": False,
      "is_unbalance": False,
      "boosting_type": 'gbdt',
      "n_jobs": 1,
      "verbosity": -1,
      "seed": 77,
}
        
fit_params= {
    'feature_name': "auto",
}

start = time.time()
estimator = LGBMClassifier(**params)

# The more CV, the better our estimate of the score
inner_cv_scores = cross_validate(estimator, X, y, 
                                 fit_params=fit_params, 
                                 cv=20, 
                                 scoring="roc_auc", 
                                 n_jobs=5, 
                                 verbose=0, 
                                 return_train_score=True)
cv_scores=inner_cv_scores['test_score'].tolist()
duration = time.time() - start

print("CV Scores:")
print(["{:0.4f}".format(cv_score) for cv_score in cv_scores])
print("CV Score mean: {:.4f} ".format(np.mean(cv_scores)))
print("CV Score range: {:0.4f} -- {:0.4f}".format(np.mean(cv_scores) - np.std(cv_scores), np.mean(cv_scores) + np.std(cv_scores)))
print("Fit times: {}".format(duration))
print("Total duration: {}".format(duration))

In [ ]:
estimator = LGBMClassifier(**params)
estimator = estimator.fit(X, y)

In [ ]:
y_pred = estimator.predict(X_test)

In [ ]:
print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, y_pred)))
print("F1 Score = {:.2f}".format(f1_score(y_test, y_pred)))
print("Log Loss = {:.2f}".format(log_loss(y_test, y_pred)))

In [ ]:
confusion_matrix(y_test, y_pred)

In [62]:
# Now try Optuna

In [81]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

def objective_dt(trial, X, y):

  # Now, define all the hyperparams we want to vary, and what values they are allowed
  # to take.
  #
  # Each trial, optuna will automatically choose values for each hyperparam.
  hyper_params = {
        "cat_l2": 10,
      "cat_smooth": 10,
      "colsample_bytree": 0.8,
      "feature_fraction_bynode": 0.8,
      "learning_rate": trial.suggest_float('learning_rate',0.01,0.05,step=0.01),
      "max_depth": trial.suggest_int('max_depth',1,12,step=2),
      "min_child_samples": trial.suggest_int('min_child_samples',53,73,step=5),
      "min_data_per_group": trial.suggest_int('min_data_per_group',50,150,step=10),
      "n_estimators": trial.suggest_int('n_estimators',100,1500,step=100),
      "num_leaves": trial.suggest_int('num_leaves',5,15,step=1),
      "path_smooth": trial.suggest_int('path_smooth',0,5,step=1),
      "reg_alpha":  trial.suggest_float('reg_alpha',0.25,0.35,step=0.025),
      "reg_lambda": trial.suggest_float('reg_lambda',0.25,0.35,step=0.025),
      "subsample_freq": trial.suggest_int('subsample_freq',1,3,step=1),
      "subsample": trial.suggest_float('subsample',0.75,0.85,step=0.01),
      "max_bin": trial.suggest_int('max_bin',27,33,step=1),
      "extra_trees": False,
      "is_unbalance": False,
      "boosting_type": 'gbdt',
      "n_jobs": 1,
      "verbosity": -1,
      "seed": 77,
  }
    

  # Use the hyperparams that optuna has chosen for this trial to create a DecisionTreeClassifier
  estimator = LGBMClassifier(**hyper_params)

  # Run CV to see how well these hyper_params do
  cv_scores = cross_val_score(estimator, X, y, cv=15, scoring="neg_log_loss")
  score = -np.mean(cv_scores)

  # Whatever we return here tells optuna how well these parameters did
  return score

In [82]:
study = optuna.create_study(direction="minimize")

[I 2023-07-27 23:04:41,543] A new study created in memory with name: no-name-c2e7ff9f-b5bc-4b89-b177-eb17a91cc9e0


In [83]:
study.optimize(lambda trial: objective_dt(trial, X, y), n_trials=100,  gc_after_trial=True)

/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:700: UserWarning: The distribution is specified by [1, 12] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 11].
  warnings.warn(
[I 2023-07-27 23:04:54,527] Trial 0 finished with value: 0.08220077415614946 and parameters: {'learning_rate': 0.02, 'max_depth': 7, 'min_child_samples': 58, 'min_data_per_group': 100, 'n_estimators': 900, 'num_leaves': 12, 'path_smooth': 0, 'reg_alpha': 0.275, 'reg_lambda': 0.3, 'subsample_freq': 3, 'subsample': 0.75, 'max_bin': 30}. Best is trial 0 with value: 0.08220077415614946.
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:700: UserWarning: The distribution is specified by [1, 12] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 11].
  warnings.warn(
[I 2023-07-27 23:05:10,757] Trial 1 finished with value: 0.07459023782229002 and parameters: {'learning_rate': 0.03, 'max_depth': 9, 'min_child_samples': 53, 'min

In [84]:
study.best_params

{'learning_rate': 0.04,
 'max_depth': 11,
 'min_child_samples': 63,
 'min_data_per_group': 100,
 'n_estimators': 1200,
 'num_leaves': 11,
 'path_smooth': 3,
 'reg_alpha': 0.25,
 'reg_lambda': 0.35,
 'subsample_freq': 2,
 'subsample': 0.85,
 'max_bin': 28}

In [85]:
study.best_value

0.06708355837520112

In [86]:
estimator = LGBMClassifier(**study.best_params)
estimator.fit(X,y)

LGBMClassifier(learning_rate=0.04, max_bin=28, max_depth=11,
               min_child_samples=63, min_data_per_group=100, n_estimators=1200,
               num_leaves=11, path_smooth=3, reg_alpha=0.25, reg_lambda=0.35,
               subsample=0.85, subsample_freq=2)

In [87]:
y_pred = estimator.predict(X_test)

In [88]:
print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, y_pred)))
print("F1 Score = {:.2f}".format(f1_score(y_test, y_pred)))
print("Log Loss = {:.2f}".format(log_loss(y_test, y_pred)))

Accuracy = 0.98
Kappa = 0.94
F1 Score = 0.95
Log Loss = 0.58


In [89]:
confusion_matrix(y_test, y_pred)

array([[101,   0],
       [  2,  21]])

In [ ]:
df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv',header='infer')
X_test = df

df['EJ_A'] = np.where(df['EJ'] == 'A',1,0)
df['EJ_B'] = np.where(df['EJ'] == 'B',1,0)
df = df.drop(['EJ'],axis=1)

In [ ]:
X_test.head()

In [ ]:
X_test_2 = df.drop(['Id'],axis=1)

In [ ]:
X_test_2.columns.unique()

In [ ]:
X_train.columns.unique()

In [ ]:
X_test_2.head()

In [ ]:
def rounder(x):
    return round(x,0)

for x in X_test_2.columns:
    if x == 'EJ_A' or x == 'EJ_B':
        imputer = imputer.fit(X_test_2[[x]])
        X_test_2[x + "_Imputed"] = imputer.transform(X_test_2[[x]])
        X_test_2 = X_test_2.drop([x],axis=1)
        X_test_2[x + '_Imputed'] = X_test_2[x + '_Imputed'].map(rounder)
    else:
        imputer = imputer.fit(X_test_2[[x]])
        X_test_2[x + "_Imputed"] = imputer.transform(X_test_2[[x]])
        X_test_2 = X_test_2.drop([x],axis=1)

In [ ]:
X_test_2.head()

In [ ]:
X_test_2['GF_Imputed_logged'] = X_test_2['GF_Imputed'].map(logger)
X_test_2['GI_Imputed_logged'] = X_test_2['GI_Imputed'].map(logger)
X_test_2['FL_Imputed_logged'] = X_test_2['FL_Imputed'].map(logger)
X_test_2['FE_Imputed_logged'] = X_test_2['FE_Imputed'].map(logger)
X_test_2['EE_Imputed_logged'] = X_test_2['EE_Imputed'].map(logger)
X_test_2['EB_Imputed_logged'] = X_test_2['EB_Imputed'].map(logger)
X_test_2['DI_Imputed_logged'] = X_test_2['DI_Imputed'].map(logger)
X_test_2['DE_Imputed_logged'] = X_test_2['DE_Imputed'].map(logger)
X_test_2['CB_Imputed_logged'] = X_test_2['CB_Imputed'].map(logger)
X_test_2['AM_Imputed_logged'] = X_test_2['AM_Imputed'].map(logger)
X_test_2['AH_Imputed_logged'] = X_test_2['AH_Imputed'].map(logger)
X_test_2['BP_Imputed_logged'] = X_test_2['BP_Imputed'].map(logger)
X_test_2['BR_Imputed_logged'] = X_test_2['BR_Imputed'].map(logger)
X_test_2['BZ_Imputed_logged'] = X_test_2['BZ_Imputed'].map(logger)
X_test_2['CC_Imputed_logged'] = X_test_2['CC_Imputed'].map(logger)
X_test_2['CD_Imputed_logged'] = X_test_2['CD _Imputed'].map(logger)
X_test_2['CF_Imputed_logged'] = X_test_2['CF_Imputed'].map(logger)
X_test_2['CH_Imputed_logged'] = X_test_2['CH_Imputed'].map(logger)
X_test_2['CL_Imputed_logged'] = X_test_2['CL_Imputed'].map(logger)
X_test_2['DL_Imputed_logged'] = X_test_2['DL_Imputed'].map(logger)
X_test_2['DU_Imputed_logged'] = X_test_2['DU_Imputed'].map(logger)
X_test_2['DV_Imputed_logged'] = X_test_2['DV_Imputed'].map(logger)
X_test_2['DY_Imputed_logged'] = X_test_2['DY_Imputed'].map(logger)
X_test_2['EG_Imputed_logged'] = X_test_2['EG_Imputed'].map(logger)
X_test_2['EH_Imputed_logged'] = X_test_2['EH_Imputed'].map(logger)
X_test_2['EP_Imputed_logged'] = X_test_2['EP_Imputed'].map(logger)
X_test_2['EU_Imputed_logged'] = X_test_2['EU_Imputed'].map(logger)
X_test_2['GL_Imputed_binary'] = np.where(X_test_2['GL_Imputed'] >= X_test_2['GL_Imputed'].mean(),1,0)
X_test_2 = X_test_2.drop('GL_Imputed',axis=1)
X_test_2['FC_Imputed_logged'] = X_test_2['FC_Imputed'].map(logger)
X_test_2['FD_Imputed_logged'] = X_test_2['FD _Imputed'].map(logger)
X_test_2['FE_Imputed_logged'] = X_test_2['FE_Imputed'].map(logger)
X_test_2['FL_Imputed_logged'] = X_test_2['FL_Imputed'].map(logger)
X_test_2['FR_Imputed_logged'] = X_test_2['FR_Imputed'].map(logger)
X_test_2['FS_Imputed_logged'] = X_test_2['FS_Imputed'].map(logger)
X_test_2['FL_Imputed_logged'] = X_test_2['FL_Imputed'].map(logger)
X_test_2['GE_Imputed_logged'] = X_test_2['GE_Imputed'].map(logger)
X_test_2['GF_Imputed_logged'] = X_test_2['GF_Imputed'].map(logger)
X_test_2['GI_Imputed_logged'] = X_test_2['GI_Imputed'].map(logger)
X_test_2['GB_Imputed_logged'] = X_test_2['GB_Imputed'].map(logger)

X_test_2['AX_Imputed_rooted'] = X_test_2['AX_Imputed'].map(rooter)
X_test_2['AZ_Imputed_rooted'] = X_test_2['AZ_Imputed'].map(rooter)
X_test_2['BD_Imputed_rooted'] = X_test_2['BD _Imputed'].map(rooter)
X_test_2['BN_Imputed_rooted'] = X_test_2['BN_Imputed'].map(rooter)
X_test_2['CR_Imputed_rooted'] = X_test_2['CR_Imputed'].map(rooter)
X_test_2['CS_Imputed_rooted'] = X_test_2['CS_Imputed'].map(rooter)
X_test_2['CU_Imputed_rooted'] = X_test_2['CU_Imputed'].map(rooter)
X_test_2['DA_Imputed_rooted'] = X_test_2['DA_Imputed'].map(rooter)
X_test_2['DH_Imputed_rooted'] = X_test_2['DH_Imputed'].map(rooter)
X_test_2['DN_Imputed_rooted'] = X_test_2['DN_Imputed'].map(rooter)
X_test_2['FI_Imputed_rooted'] = X_test_2['FI_Imputed'].map(rooter)
X_test_2['GH_Imputed_rooted'] = X_test_2['GH_Imputed'].map(rooter)

In [ ]:
y_pred = estimator.predict_proba(X_test_2)

In [ ]:
y_pred

In [ ]:
X_test_2

In [ ]:
submission = pd.DataFrame(X_test['Id'], columns=['Id'])

submission[['class_0','class_1']] = y_pred
submission.to_csv('submission.csv', index=False) 